<center><h2>Topic: IT Ticket Classification and Assignment</h2></center>

<center><h3>Student Name: Yashwanth Muddanna Hanumantharaya</h3></center>
<center><h3>Student ID: 23284986</h3></center>
<center><h4>Course: MSc in Artificial Intelligence</h3></center>
<center><h4>Module: Intelligent Agents and Process Automation</h3></center>

## Importing the required libraries

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

## Data loading and EDA

In [24]:
# Load dataset
df = pd.read_csv("sample_it_tickets_new.csv")
df["text"] = df["subject"] + " " + df["description"]

In [25]:
df.head()

,subject,description,category,text
0,Blocked from system,Please reset my Windows login password.,Access,Blocked from system Please reset my Windows lo...
1,Unable to install software,I'm unable to install the software needed for ...,Software,Unable to install software I'm unable to insta...
2,Network drive unavailable,Internet is very slow and affects meetings.,Network,Network drive unavailable Internet is very slo...
3,Software update failed,License has expired and I can't open the program.,Software,Software update failed License has expired and...
4,Keyboard not working,Keyboard keys are unresponsive.,Hardware,Keyboard not working Keyboard keys are unrespo...


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   subject      100 non-null    object
 1   description  100 non-null    object
 2   category     100 non-null    object
 3   text         100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


## Data Split and Pipeline

In [27]:
# Train & test split
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["category"], test_size=0.3, random_state=42)

In [28]:
# Text classification pipeline using Logistic Regression
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer()), #TfidfVectorizer is used to convert the text data into numerical form
    ("clf", LogisticRegression(max_iter=1000))
])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LogisticRegression(max_iter=1000))])

In [29]:
# Assignment map
assignment_map = {
    "Hardware": "Level 2 Support",
    "Software": "Application Support",
    "Network": "Network Admins",
    "Access": "IT Security Team",
    "Printer": "Facilities IT"
}

In [30]:
# Function to classify and assign ticket
def classify_and_assign(subject, description):
    text = subject + " " + description
    category = pipeline.predict([text])[0]
    assigned_team = assignment_map.get(category, "General IT Support")
    return category, assigned_team

## Testing with 1 ticket

In [31]:
new_ticket = {
    "subject": "Cannot print documents",
    "description": "The printer in Room 203 is not printing anything even though it shows as online."
}

In [32]:
# Predict and assign
category, team = classify_and_assign(new_ticket["subject"], new_ticket["description"])

# Running the pipeline on the new_ticket
print(f"Predicted Category: {category}")
print(f"Assigned Team: {team}")


Predicted Category: Printer
Assigned Team: Facilities IT


## Testing with 100 ticket

In [33]:
result = df[['subject', 'description']].to_dict(orient='records')
print(len(result))

for ticket in result:
    category, team = classify_and_assign(ticket["subject"], ticket["description"])
    print(f"Subject: {ticket['subject']}")
    print(f"Predicted Category: {category}")
    print(f"Assigned Team: {team}")
    print("-" * 50)

100
Subject: Blocked from system
Predicted Category: Access
Assigned Team: IT Security Team
--------------------------------------------------
Subject: Unable to install software
Predicted Category: Software
Assigned Team: Application Support
--------------------------------------------------
Subject: Network drive unavailable
Predicted Category: Network
Assigned Team: Network Admins
--------------------------------------------------
Subject: Software update failed
Predicted Category: Software
Assigned Team: Application Support
--------------------------------------------------
Subject: Keyboard not working
Predicted Category: Hardware
Assigned Team: Level 2 Support
--------------------------------------------------
Subject: Blocked from system
Predicted Category: Access
Assigned Team: IT Security Team
--------------------------------------------------
Subject: System overheating
Predicted Category: Hardware
Assigned Team: Level 2 Support
-----------------------------------------------

Subject: 2FA issue
Predicted Category: Access
Assigned Team: IT Security Team
--------------------------------------------------
Subject: Low toner warning
Predicted Category: Printer
Assigned Team: Facilities IT
--------------------------------------------------
Subject: App crash issue
Predicted Category: Software
Assigned Team: Application Support
--------------------------------------------------
Subject: Battery not charging
Predicted Category: Hardware
Assigned Team: Level 2 Support
--------------------------------------------------
Subject: Unable to install software
Predicted Category: Software
Assigned Team: Application Support
--------------------------------------------------
Subject: Low toner warning
Predicted Category: Printer
Assigned Team: Facilities IT
--------------------------------------------------
Subject: Email account locked
Predicted Category: Access
Assigned Team: IT Security Team
--------------------------------------------------
Subject: Blocked from system


In [34]:
results = []
for ticket in result:
    category, team = classify_and_assign(ticket["subject"], ticket["description"])
    results.append({
        "Subject": ticket["subject"],
        "Description": ticket["description"],
        "Predicted Category": category,
        "Assigned Team": team
    })

# Convert to DataFrame and save to CSV
df_results = pd.DataFrame(results)
df_results.to_csv("classified_tickets_output.csv", index=False)

## Additional Step for Future Work

In [13]:
# Email set-up
sender_email = "yashwanthmhedu@gmail.com"
sender_password = "rrqq vnyf yhmk rlva"  # Use app-specific password for Gmail
smtp_server = "smtp.gmail.com"
smtp_port = 587

# Team eamils taken smaples 
team_emails = {
    "Hardware": "hardware_support@example.com",
    "Software": "software_team@example.com",
    "Network": "network_admins@example.com",
    "Access": "it_security@example.com",
    "Printer": "printer_support@example.com"
}

def send_ticket_email(subject, description, category):
    receiver_email = team_emails.get(category, "general_support@example.com")
    
    # Email body 
    message = MIMEMultipart("alternative")
    message["Subject"] = f"New IT Ticket - {category}"
    message["From"] = sender_email
    message["To"] = receiver_email
    
    text = f"""\
Hi {category} Support Team,

A new ticket has been assigned to you.

Subject: {subject}
Description: {description}
Category: {category}

Please address this ticket as soon as possible.

Regards,
IT Automation System
"""
    message.attach(MIMEText(text, "plain"))
    
    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, message.as_string())
        server.quit()
        print(f"Email sent to {receiver_email}")
    except Exception as e:
        print("Failed to send email:", e)

# Example usage after classification
ticket_subject = "Cannot connect to WiFi"
ticket_description = "User reports no internet in the 3rd-floor meeting room."
predicted_category = "Network"

send_ticket_email(ticket_subject, ticket_description, predicted_category)


Email sent to network_admins@example.com


## Testing with more emails

In [14]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Your email credentials (use app password if using Gmail with 2FA)
sender_email = "yashwanthmhedu@gmail.com"
sender_password = "rrqq vnyf yhmk rlva"
smtp_server = "smtp.gmail.com"
smtp_port = 587

# Mapping categories to team emails
team_emails = {
    "Hardware": "yashwanthmhedu@example.com",
    "Software": "yashwanthmhedu@example.com",
    "Network": "yashwanthmhedu@example.com",
    "Access": "yashwanthmhedu@example.com",
    "Printer": "yashwanthmhedu@example.com"
}

# Sample 15 tickets
tickets = [
    {"subject": "Laptop won't start", "description": "No response on power button.", "category": "Hardware"},
    {"subject": "WiFi keeps disconnecting", "description": "Losing connection every 10 minutes.", "category": "Network"},
    {"subject": "Need Adobe Photoshop", "description": "Install Photoshop for marketing team.", "category": "Software"},
    {"subject": "Request access to folder", "description": "Need access to HR documents.", "category": "Access"},
    {"subject": "Printer paper jam", "description": "Paper stuck in tray 2.", "category": "Printer"},
    {"subject": "Battery drains fast", "description": "Battery lasts only 30 minutes.", "category": "Hardware"},
    {"subject": "VPN not working", "description": "Can't connect from home.", "category": "Network"},
    {"subject": "Outlook crashes", "description": "Outlook closes after start.", "category": "Software"},
    {"subject": "2FA issue", "description": "Code not received on phone.", "category": "Access"},
    {"subject": "Toner alert", "description": "Printer says toner low.", "category": "Printer"},
    {"subject": "Laptop overheating", "description": "Very hot under normal use.", "category": "Hardware"},
    {"subject": "Slow internet", "description": "Internet lags on video calls.", "category": "Network"},
    {"subject": "Excel not opening", "description": "Clicking icon does nothing.", "category": "Software"},
    {"subject": "Locked out", "description": "Forgot Windows password.", "category": "Access"},
    {"subject": "Printer queue stuck", "description": "Jobs won't print.", "category": "Printer"}
]

# Function to send email
def send_ticket_email(subject, description, category):
    receiver_email = team_emails.get(category, "default_team@example.com")
    msg = MIMEMultipart()
    msg["Subject"] = f"[New Ticket] {subject}"
    msg["From"] = sender_email
    msg["To"] = receiver_email
    body = f"""\
Hello {category} Team,

You have a new support ticket assigned.

Subject: {subject}
Description: {description}
Category: {category}

Please address this issue at your earliest convenience.

Best regards,
IT Automation System
"""
    msg.attach(MIMEText(body, "plain"))
    
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, receiver_email, msg.as_string())
        print(f"Email sent to {receiver_email}")
    except Exception as e:
        print(f"Failed to send email for '{subject}': {e}")

# Send emails for all 15 tickets
for ticket in tickets:
    send_ticket_email(ticket["subject"], ticket["description"], ticket["category"])

Email sent to yashwanthmhedu@example.com
Email sent to yashwanthmhedu@example.com
Email sent to yashwanthmhedu@example.com
Email sent to yashwanthmhedu@example.com
Email sent to yashwanthmhedu@example.com
Email sent to yashwanthmhedu@example.com
Email sent to yashwanthmhedu@example.com
Email sent to yashwanthmhedu@example.com
Email sent to yashwanthmhedu@example.com
Email sent to yashwanthmhedu@example.com
Email sent to yashwanthmhedu@example.com
Email sent to yashwanthmhedu@example.com
Email sent to yashwanthmhedu@example.com
Email sent to yashwanthmhedu@example.com
Email sent to yashwanthmhedu@example.com
